# BOW & TF-IDF - recitation

``
In this recitation, you will experience with implementing "tf-idf document vectorizer", and see how it can be use to solve practical problem.
``

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from functools import reduce
import operator

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

```What we going to do here is to solve the next problem:
Given a set of articles writen by different authors, we want to decide the "authorship" of a new article writen by one of the authors.```

In [ ]:
## can be found in: https://drive.google.com/open?id=1Ni5Rb_q9gdCIGPW2i1oyH2a5G823fOio
df = pd.read_csv('data/Gungor_2018_VictorianAuthorAttribution.csv')

In [ ]:
## we have articles of 10 different authors
df.groupby('author').agg(article_count=pd.NamedAgg(column="text", aggfunc="count"))

## "bag of words" (BOW)

wikipedia link: https://en.wikipedia.org/wiki/Bag-of-words_model

```Bag of words is a way to represent a document (or a sentance) as the "counts" of the words in it.```

``That way, if we have the vocabulary of the language (all the words in the language) and we agree on some order of thses words (say ["television", "mouse", ...]), we can represent any socument as a vector that each value in it is the count of the corresponding word in the document.``

example:

```doc       = "John likes to watch movies. Mary likes movies too.
vocabulary = ["John, "likes", "to", "watch", "movies", "Mary", "too", "book"].
=> bow = [1, 2, 1, 1, 2, 1, 1, 0].```

With this representation we can measure "distance" between documents.

```The most way to do it is "cosine similaruty". For vector v1, and vector v2, we define "cosine similaruty" as:```

$$similarity(v1, v2) = \frac{<v1, w2>}{\lVert v1 \rVert \cdot \lVert v2 \rVert}$$

```And it is the value of``` $cos(\theta)$, ```where``` $\theta$ ```is the angle between v1 and v2. (big similarity(v1, v2) means v1 and v2 are close to each other)```

(Explanation can be found in: https://en.wikipedia.org/wiki/Cosine_similarity)

``Lets make our articles to be in BOW shape:``

In [ ]:
def clean_words(document):
    
    doc_strings = document.split(' ')
    doc_strings = [i.strip() for i in doc_strings]
    doc_strings = [i for i in doc_strings if i != '']
    
    return doc_strings

In [ ]:
documents = df['text'].apply(clean_words).tolist()
authors = df['author'].values

``Lets spit the data into train and test. We remember that the meaning of the train and test is: data we have (train), and data we will see in the future (test).``

``Because we have only the train data when we "learn", the words we consider for the model are only the words we see in the train.``

In [ ]:
train_docs, test_docs, train_authors, test_authors = train_test_split(documents, authors)

In [ ]:
all_words = reduce(operator.iconcat, train_docs, [])
vocabulary = np.array(list(set(all_words))) # the vocabulary composed only from the words in the train

Now we create vectorizer that given the vocabulary, can turn documents to BOW vectors:

In [ ]:
def create_document_vectorizer(words):
    
    indexer = {w:i for i,w in enumerate(words)}
    
    def vectorizer(document):
        N, vocab_size = len(documents), len(words)
        vec = np.zeros(vocab_size)
        for w in document:
            if w in indexer: # if we dont have the word in the vocabulary we not counting it
                vec[indexer[w]] += 1
        return vec
    
    return vectorizer

In [ ]:
vectorizer = create_document_vectorizer(vocabulary)

In [ ]:
train_bow_vectors = np.vstack([vectorizer(doc) for doc in train_docs])
test_bow_vectors = np.vstack([vectorizer(doc) for doc in test_docs])

In [ ]:
def pairwise_cosine_similarity(vecs1, vecs2):
    """
    vecs1: N1 x M dim matrix (N1 vectors with dim M)
    vecs2: N2 x M dim matrix (N2 vectors with dim M)
    
    return: N1 x N2 dim matrix "scores", where scores[i,j] = cosine_similarity(vecs1[i], vecs2[j])
    """
    
    scores = # answer (can check it against "sklearn.metrics.pairwise.cosine_similarity")
    
    return scores

```In order to decise the author of every document in the test, we will calculate the cosine similarity between every doc in the test to every doc in the train.
We will say that the author of a document in the test is the author of its "most similar document in the train". (highest score)```

In [ ]:
scores = pairwise_cosine_similarity(test_bow_vectors, train_bow_vectors)

In [ ]:
predicted_authors = train_authors[scores.argmax(axis=1)]

In [ ]:
print('accuracy: ', np.mean(predicted_authors == test_authors))

In [ ]:
print('confision matrix:')
confusion_matrix(test_authors, predicted_authors)

```Not bad hah? Lets see how can we improve it.```

## tf-idf (term frequency–inverse document frequency)

wikipedia link: https://en.wikipedia.org/wiki/Tf%E2%80%93idf

```Main problem of "bag of words" approach (in addition to the lost of "words order"), is that we give significant wight to "stop words.```

<b>stop words:</b> ```are the most common words in the language (i.e. it, I, was, to, ...), and they can be found in almost all documents. Therfore, they cannot help us "distinguish" between documents, and usualy their value in the BOW vector is very high compared to other words.```

```tf-idf come to solve this problem by giving "score" to each word that represent: how much this word "distinguish" the documents we have.```

Say we $N$ documents $\{doc_1, ..., doc_N\}$, and vocabulary $\{w_1, ..., w_M\}$.

We define $n_w$ = "in how many documents w appears".

Lets define the "idf" (inverse document frequency) score for a word $w$ as: $idf(w) = log(\frac{N}{n_w + 1})$

```(low idf value means "stop word", and high as distinguishing word)```


```We now can improve our BOW representation of some new document by multiplying every value in the BOW vector with the corresponting idf score
(td-idf[i] = bow[i]*idf[i]), and get tf-idf representation.```



<b>Remark:</b>

```The training process of this tf-idf "model" is the calculation of the idf scores.
(And there are more ways to calculate idf scores)```

In [ ]:
def culc_idf_scores(bow_vectors):
    """
    bow_vectors: matrix of shape (num documents x vocabulary size)
    
    return: idf score for every word in the vocabulary
    """
    
    idf = # answer
    
    return idf

In [ ]:
idf = culc_idf_scores(train_bow_vectors)

```Lets see which words got low scores:```

In [ ]:
vocabulary[np.argsort(idf)[:50]]

```Lets see which words got high scores:```

In [ ]:
vocabulary[np.argsort(idf)[::-1][:50]]

In [ ]:
train_tfidf_vectors = train_bow_vectors * idf
test_tfidf_vectors  = test_bow_vectors * idf

In [ ]:
scores = pairwise_cosine_similarity(test_tfidf_vectors, train_tfidf_vectors)

In [ ]:
predicted_authors = train_authors[scores.argmax(axis=1)]

In [ ]:
print('accuracy: ', np.mean(predicted_authors == test_authors))

In [ ]:
print('confision matrix:')
confusion_matrix(test_authors, predicted_authors)

## Bonus

```Make a vector for every author, as the mean of all his tf-idf vectors in the train.
Try to compare the test tf-idf vectors only to those vectors now instead of all the vectors.
How the accuracy now? how is the computation time?```

```Now do the same, but instead of the one "mean" vector for each author, find 3 vectors for each author as the cluster centers of KMeans(k=3) on his train tf-idf vectors. (try low n_init to reduce run time)
Is it more helpful?```